In [2]:
import pandas as pd
from IPython.core.debugger import set_trace
import json
import re
import os
import time
import requests
from twython import Twython, TwythonRateLimitError, TwythonError
from app.models import Brand, Profile, Hashtag
from custom_user.models import EmailUser
from tqdm.autonotebook import tqdm
tqdm.pandas()

# Scraping

In [79]:
df = pd.read_excel('../texcrapy/keywords and logos.xlsx', 
                   sheet_name='20190304')[['shortname','kw_supporter','kw_supported','keywords']][3:];df
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

qry_base = {row.shortname:_or(row.keywords) for row in df.itertuples()}
supporters = df.shortname[df.kw_supporter==True]
qry_sup = ' OR '.join([qry_base[sup] for sup in supporters]); qry_sup
qry = {row.shortname: '(' + qry_base[row.shortname] + ') AND (' + qry_sup + ')' if row.kw_supported==True else qry_base[row.shortname] for row in df.itertuples()}

In [58]:
%%time
what = ['id']#, 'likes', 'text', 'timestamp', 'user']
texcrapy.scrap(qry, what=what, lang='ko', start='2019-01-01', end='2019-06-20', download_to='scrapped/getch')

NameError: name 'texcrapy' is not defined

In [3]:
from glob import glob
glob_pattern = os.path.join('../texcrapy/scrapped/twitter_getch', '*')
fnames = glob(glob_pattern)
# fnames = ['scrapped/twitter_getch/' + fname for fname in os.listdir('../texcrapy/scrapped/twitter_getch')]

In [4]:
tweet_ids = {}
for fname in tqdm(fnames[:]):
    with open(fname, encoding='UTF-8-sig') as f:
        for item, docs in json.load(f).items():
            if len(docs) > 0:
                tweet_ids[item] = set()
                for doc in docs: tweet_ids[item].add(doc['id'])

In [5]:
tweet_ids

{'1017alyx9sm': {'1087988510936784901'},
 '8seconds': {'1085168379009220608',
  '1093667804216098816',
  '1096278432579608576',
  '1108748381331963905',
  '1109288688847151105',
  '1113057180905684997',
  '1123169729940602880',
  '1124151204148174850',
  '1124151878550343681',
  '1124510975959240706',
  '1124904859549995008',
  '1126064951922003969',
  '1133544959624318976',
  '1135748910222151680',
  '1138329340733603840',
  '1138403305065893888',
  '1139037798483210240',
  '1139794453504225280',
  '1140818488862240768'},
 '99percentis': {'1109096052504711169',
  '1109295920456204289',
  '1109296009945841664',
  '1109296056607494148'},
 'aape': {'1115559946127577088', '1131475096323989504'},
 'acnestudios': {'1080660463484952576',
  '1086058203454959616',
  '1087686766155321344',
  '1090274927087804416',
  '1091745098084143105',
  '1098752443934896129',
  '1103542668489965568',
  '1105355730511572993',
  '1110905237966147584',
  '1114712618910998528',
  '1119152943221993473',
  '11191

# 트위터 크롤링 (by ID)

In [11]:
def get_hashtags(tweet):
    _hashtags = tweet['entities']['hashtags']
    
    if len(_hashtags) > 0:
        return {ht['text'] for ht in _hashtags}
    else:
        return set()
                

def get_user(tweet):
    _user = tweet['user']
    
    user = {}
    user['user'] = _user['screen_name']

    if 'profile_image_url' in _user:
        user['profile_image'] = _user['profile_image_url']
            
    return {_user['id']:user}
        
    
def get_feed_image(tweet):
    entities = tweet['entities']
    
    if 'media' in entities:
        media = entities['media']
        
        if len(media) > 0:
            _media = media[0]
            
            if 'media_url' in _media:
                return _media['media_url']
            
    return None
    

def get_feed(tweet, bname, hashtags):
    feed = {
        'membership': bname, 
        'author': tweet['user']['screen_name'], 
        'created_at': tweet['created_at'], 
        'nlikes': tweet['favorite_count'], 
        'content': tweet['text'],
        'hashtags': list(hashtags),
    }
    
    feed_image = get_feed_image(tweet)
    
    if feed_image is not None:
        feed['feed_image'] = feed_image
        
    return feed

In [12]:
CONSUMER_KEY = 'gFeP16lpEBKux4K6GO2LfLEWI'
CONSUMER_SECRET = '38yzPyI8vqUqrCV0RLc1ugl8Se1pYA2Nlderk9Rj3DUaE5VTpA'
OAUTH_TOKEN = '1085907708413267970-nyk4nvgG9miNTqRmg4jkFnmZfqW254'
OAUTH_TOKEN_SECRET = 'ty2TIFFSorxmMKMq0VlqcIWrgqpJqdHBW9z3ye0J4BeL7'
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

feeds = []
users = dict()
hashtags = set()

for bname, ids in tqdm(list(tweet_ids.items())[:2]):
    for tw_id in tqdm(ids, leave=False):
        try:
            tw = twitter.show_status(id=tw_id)
            _hashtags = get_hashtags(tw)
            _user = get_user(tw)
            _feed = get_feed(tw, bname, _hashtags)
            
            hashtags |= _hashtags
            #users.add(frozenset(_user.items()))
            users.update(_user)
            feeds.append(_feed)
                    
                    
        except TwythonRateLimitError as error:
            remainder = float(twitter.get_lastfunction_header(header='x-rate-limit-reset')) - time.time()
            twitter.disconnect()
            time.sleep(remainder)
            twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
            continue
        
        except TwythonError as e:
            print(bname, tw_id, e)

In [8]:
users

{128813558: {'user': 'DazedKorea',
  'profile_image': 'http://pbs.twimg.com/profile_images/1130299010613358592/2z0vgQF1_normal.jpg'},
 38125688: {'user': 'agonyman',
  'profile_image': 'http://pbs.twimg.com/profile_images/1140586197909037056/MRy_Mn13_normal.jpg'},
 1123837718017073153: {'user': '8056LpjXNJdgwuj',
  'profile_image': 'http://pbs.twimg.com/profile_images/1123868952927637505/pXgIXYNZ_normal.jpg'},
 2164284031: {'user': 'fashionncom',
  'profile_image': 'http://pbs.twimg.com/profile_images/723049898157834240/PZH9sfj5_normal.jpg'},
 3230283662: {'user': 'rvd3065',
  'profile_image': 'http://pbs.twimg.com/profile_images/604481976528011264/ZqhWDujV_normal.jpg'},
 751286120177033216: {'user': 'jung_sang_won',
  'profile_image': 'http://pbs.twimg.com/profile_images/1029773819081183232/L2-i6nGr_normal.jpg'},
 586269115: {'user': 'woodz_golmu',
  'profile_image': 'http://pbs.twimg.com/profile_images/1133647436813987840/T2xj6cCa_normal.jpg'},
 98787180: {'user': 'musinsacom',
  'pr

In [9]:
hashtags

{'3000만큼사랑해',
 '4dx',
 '8SECONDS',
 '8SxME',
 '8seconds',
 'MUSINSA',
 'SPA',
 'baggies',
 'bigbang',
 'buckaroo',
 'cardigan',
 'cgv',
 'file',
 'gd',
 'gdragon',
 'goods',
 'jiyong',
 'kwonjiyong',
 'menstyle',
 'onepunchman',
 'shirts',
 'uniqlo',
 '가로수길',
 '굿즈',
 '권지용',
 '노트',
 '단독특가',
 '대리',
 '데이즈드',
 '드리미',
 '롯데시네마',
 '롱패딩',
 '명동',
 '무신사',
 '무신사TV',
 '무신사스토어',
 '무인양품',
 '브이로그',
 '블레이저',
 '빅뱅',
 '서울스토어',
 '셔츠',
 '스파오',
 '슬로건',
 '알바',
 '양도',
 '어벤져스',
 '에잇세컨즈',
 '옷',
 '완판남',
 '유니클로',
 '잠옷',
 '조승연',
 '지드래곤',
 '지디',
 '출근룩',
 '트렌치코트',
 '파일',
 '파자마',
 '판매',
 '패션',
 '한현민'}

In [13]:
feeds

[{'membership': '1017alyx9sm',
  'author': 'DazedKorea',
  'created_at': 'Wed Jan 23 08:20:49 +0000 2019',
  'nlikes': 10,
  'content': '매튜 윌리엄스의 1017 알릭스 9SM는 카이어 거버를 필두로 ‘Ex Nihilo(라틴어로 ‘무에서 벗어나다’를 의미)’란 이름의 하이테크 컬렉션을 펼쳤습니다. \n\n더 자세한 내용은 #데이즈드 인스타그램을… https://t.co/nMn8zUzio3',
  'hashtags': ['데이즈드']},
 {'membership': '8seconds',
  'author': 'agonyman',
  'created_at': 'Wed May 29 01:25:53 +0000 2019',
  'nlikes': 0,
  'content': '새벽엔 좀 선선하더만 아침부터 햇살이 좀 강하네. \n\n이럴 땐...\n\n#menstyle \n#uniqlo #cardigan \n#buckaroo #baggies \n#onepunchman #shirts… https://t.co/sHLkSodd2h',
  'hashtags': ['cardigan',
   'buckaroo',
   'menstyle',
   'onepunchman',
   'uniqlo',
   'shirts',
   'baggies']},
 {'membership': '8seconds',
  'author': '8056LpjXNJdgwuj',
  'created_at': 'Wed May 08 10:03:00 +0000 2019',
  'nlikes': 0,
  'content': '두번째 영상 업로드으 😝\n#롯데시네마 #드리미 #알바 #브이로그 #cgv #4dx #에잇세컨즈 #어벤져스 #3000만큼사랑해\n\n🍿롯데시네마 알바 브이로그 / CGV가서 4DX로 영화 어벤져스 봐요🎬 https://t.co/nmY9jGPzh7',
  'hashtags': ['4dx',
  

# Profile image 저장하기

In [16]:
for k,v in tqdm(users.items()):
    profile_dir = 'uploads/profiles_images/' + str(k)
    
    if not os.path.isdir(profile_dir):
        os.mkdir(profile_dir)
    
    profile_image = v['profile_image']
    fname = profile_dir + '/' + profile_image.split('/')[-1]

    if not os.path.isfile(fname):
        try:
            img = requests.get(profile_image).content
            with open(fname, 'wb') as f:
                f.write(img)
        except:
            print(k)

# Feed image 저장하기

In [18]:
for feed in tqdm(feeds):
    if 'feed_image' not in feed:
        continue
    
    feed_image = feed['feed_image']
    feed_memb_dir = 'uploads/feed_images/' + feed['membership']

    if not os.path.isdir(feed_memb_dir):
        os.mkdir(feed_memb_dir)
    
    feed_memb_author_dir = feed_memb_dir + '/' + feed['author']
    
    if not os.path.isdir(feed_memb_author_dir):
        os.mkdir(feed_memb_author_dir)
    
    fname = feed_memb_author_dir + '/' + feed_image.split('/')[-1]

    if not os.path.isfile(fname):
        try:
            img = requests.get(feed_image).content
            with open(fname, 'wb') as f:
                f.write(img)
        except:
            print(fname)

# 테스트유저 만들기
## 1. Custom user model (email 기반)


여기서부터 시작하자

In [17]:
dbinit_test_users = []

for k,v in tqdm(users.items()):
    dbinit_test_users.append({
        'model': 'custom_user.EmailUser', 
        'pk': k, 
        'fields': {
            'email': v['user'] + '@getch.com', 
        }
    })

In [18]:
dbinit_test_users

[{'model': 'custom_user.EmailUser',
  'pk': 128813558,
  'fields': {'email': 'DazedKorea@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 907591373548687360,
  'fields': {'email': '4buynsell@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 3230283662,
  'fields': {'email': 'rvd3065@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 156555363,
  'fields': {'email': 'i_ilyo@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 98787180,
  'fields': {'email': 'musinsacom@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 586269115,
  'fields': {'email': 'woodz_golmu@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 1054274956572295168,
  'fields': {'email': 'SeoulStore_twt@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 1136096321612345344,
  'fields': {'email': 'joggingdan@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 2164284031,
  'fields': {'email': 'fashionncom@getch.com'}},
 {'model': 'custom_user.EmailUser',
  'pk': 3

In [40]:
# dbinit_test_users = []
# users = []
# pk = 1000000

# for fname in fnames:
#     with open(fname, encoding='UTF-8-sig') as f:
#         for item, docs in json.load(f).items():
#             for doc in docs:
#                 user = doc['user']
                
#                 if user not in users:
#                     users.append(user)
                
#                     dbinit_test_users.append({
#                         'model': 'custom_user.EmailUser', 
#                         'pk': pk, 
#                         'fields': {
#                             'email': user + '@getch.com', 
#                         }
#                     })
#                     pk += 1

In [46]:
with open('dbinit_test_users.json', 'w') as f:
    json.dump(dbinit_test_users, f)

## 2. Profile model

In [19]:
dbinit_test_profiles = []

for k,v in tqdm(users.items()):
    dbinit_test_profiles.append({
        'model': 'app.profile', 
        'pk': k, 
        'fields': {
            'user': k, 
            'profile_image': str(k) + '/' + v['profile_image'].split('/')[-1], 
        }
    })

In [21]:
dbinit_test_profiles

[{'model': 'app.profile',
  'pk': 128813558,
  'fields': {'user': 128813558,
   'profile_image': '128813558/2z0vgQF1_normal.jpg'}},
 {'model': 'app.profile',
  'pk': 907591373548687360,
  'fields': {'user': 907591373548687360,
   'profile_image': '907591373548687360/byVMGAaP_normal.jpg'}},
 {'model': 'app.profile',
  'pk': 3230283662,
  'fields': {'user': 3230283662,
   'profile_image': '3230283662/ZqhWDujV_normal.jpg'}},
 {'model': 'app.profile',
  'pk': 156555363,
  'fields': {'user': 156555363,
   'profile_image': '156555363/UYe5Z8JL_normal.jpg'}},
 {'model': 'app.profile',
  'pk': 98787180,
  'fields': {'user': 98787180,
   'profile_image': '98787180/h6WKfU8c_normal.jpg'}},
 {'model': 'app.profile',
  'pk': 586269115,
  'fields': {'user': 586269115,
   'profile_image': '586269115/T2xj6cCa_normal.jpg'}},
 {'model': 'app.profile',
  'pk': 1054274956572295168,
  'fields': {'user': 1054274956572295168,
   'profile_image': '1054274956572295168/4l3C6QTh_normal.png'}},
 {'model': 'app.pro

In [50]:
with open('dbinit_test_profiles.json', 'w') as f:
    json.dump(dbinit_test_profiles, f)

# Hashtag model

In [17]:
# def get_hashtags(text):
#     LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
#     RT = '((?: rt)|(?:^rt))[^ @]?'
#     EMOJI = r'[\U00010000-\U0010ffff]'
#     DOTS = '…'
#     LONG_BLANK = r'[ ]+'
#     SPECIALS = r'([^ a-zA-Z0-9_\u3131-\u3163\uac00-\ud7a3]+)|([ㄱ-ㅣ]+)'
    
#     # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
#     # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
#     URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-zA-Z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
#     URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
#     URL = '|'.join((URL1, URL2))
    
#     HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
#     MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
#     #text = text.lower()
#     text = re.sub(URL, ' ', text)
#     text = re.sub(MENTION, ' ', text)                
#     text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS)), ' ', text)
#     #text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), ' ', text)
#     #set_trace()
#     return set(re.findall(HASHTAG, text))

In [19]:
dbinit_hashtag = []
# hashtags = set()
# pk = 0

# for fname in fnames:
#     with open(fname, encoding='UTF-8-sig') as f:
#         for item, docs in json.load(f).items():
#             for doc in docs:
#                 hashtags |= get_hashtags(doc['text'])
                
for pk, tag in enumerate(hashtags):
    dbinit_hashtag.append({
        'model': 'app.hashtag', 
        'pk': pk, 
        'fields': {
            'hashtag': tag, 
        }
    })

In [22]:
dbinit_hashtag

[{'model': 'app.hashtag', 'pk': 0, 'fields': {'hashtag': '롯데시네마'}},
 {'model': 'app.hashtag', 'pk': 1, 'fields': {'hashtag': 'goods'}},
 {'model': 'app.hashtag', 'pk': 2, 'fields': {'hashtag': 'file'}},
 {'model': 'app.hashtag', 'pk': 3, 'fields': {'hashtag': 'buckaroo'}},
 {'model': 'app.hashtag', 'pk': 4, 'fields': {'hashtag': '명동'}},
 {'model': 'app.hashtag', 'pk': 5, 'fields': {'hashtag': '8seconds'}},
 {'model': 'app.hashtag', 'pk': 6, 'fields': {'hashtag': '알바'}},
 {'model': 'app.hashtag', 'pk': 7, 'fields': {'hashtag': '잠옷'}},
 {'model': 'app.hashtag', 'pk': 8, 'fields': {'hashtag': '굿즈'}},
 {'model': 'app.hashtag', 'pk': 9, 'fields': {'hashtag': '대리'}},
 {'model': 'app.hashtag', 'pk': 10, 'fields': {'hashtag': '지디'}},
 {'model': 'app.hashtag', 'pk': 11, 'fields': {'hashtag': '무신사'}},
 {'model': 'app.hashtag', 'pk': 12, 'fields': {'hashtag': '파자마'}},
 {'model': 'app.hashtag', 'pk': 13, 'fields': {'hashtag': '노트'}},
 {'model': 'app.hashtag', 'pk': 14, 'fields': {'hashtag': '조승연'}

In [73]:
with open('dbinit_hashtag.json', 'w') as f:
    json.dump(dbinit_hashtag, f)

# Feed model

In [24]:
from app.models import Brand, Profile, Hashtag

In [ ]:
dbinit_feed = []

for pk, feed in enumerate(feeds):
    membership_pk = Brand.objects.get(name=feed['membership']).pk
    author_pk = Profile.objects.get(user__email=feed['author']+'@getch.com').pk
    hashtag_pks = list(Hashtag.objects.filter(hashtag__in=feed['hashtags']).values_list('pk', flat=True))
    feed_image = feed['membership'] + '/' + feed['author'] + '/' + feed['feed_image'].split('/')[-1]
    
    dbinit_feed.append({
        'model': 'app.feed', 
        'pk': pk, 
        'fields': {
            'membership': membership_pk, 
            'author': author_pk, 
            'timestamp': feed['created_at'], 
            'nlikes': feed['nlikes'], 
            'content': feed['content'], 
            'hashtags': hashtag_pks,
            'feed_image': feed_image
        }
    })    

In [3]:
from twython import Twython
from app.models import Brand, Profile, Hashtag, Feed
from custom_user.models import EmailUser

In [38]:
dbinit_feed = []
pk = 0

for fname in tqdm_notebook(fnames):
    with open(fname, encoding='UTF-8-sig') as f:
        for item, docs in json.load(f).items():
            for doc in docs:
                _hashtags = get_hashtags(doc['text'])
                hashtag_pks = list(Hashtag.objects.filter(hashtag__in=_hashtags).values_list('pk', flat=True))
                membership_pk = Brand.objects.get(name=item).pk
                author_pk = Profile.objects.get(user__email=doc['user']+'@getch.com').pk
                
                dbinit_feed.append({
                    'model': 'app.feed', 
                    'pk': pk, 
                    'fields': {
                        'membership': membership_pk, 
                        'author': author_pk, 
                        'timestamp': doc['timestamp'], 
                        'nlikes': doc['likes'], 
                        'content': doc['text'], 
                        'hashtags': hashtag_pks
                    }
                })
                pk += 1

In [39]:
with open('dbinit_feed.json', 'w') as f:
    json.dump(dbinit_feed, f)

In [29]:
{ht['text'] for ht in twitter.show_status(id='1112390453813338112')['entities']['hashtags']}

{'BTSISCOMING', 'MAP_OF_THE_SOUL_PERSONA'}

In [64]:
twitter.show_status(id='1142599021342707713')

{'created_at': 'Sun Jun 23 01:03:29 +0000 2019',
 'id': 1142599021342707713,
 'id_str': '1142599021342707713',
 'text': 'How Owning A Tesla Model 3 For 1 Year Has Changed My\xa0Life https://t.co/BXvlkO68O5 https://t.co/UJbFZk6FWm',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/BXvlkO68O5',
    'expanded_url': 'https://cleantechnica.com/2019/06/22/how-owning-a-tesla-model-3-for-1-year-has-changed-my-life/',
    'display_url': 'cleantechnica.com/2019/06/22/how…',
    'indices': [58, 81]}],
  'media': [{'id': 1142599019438493697,
    'id_str': '1142599019438493697',
    'indices': [82, 105],
    'media_url': 'http://pbs.twimg.com/media/D9tTwSJXUAEjiFU.png',
    'media_url_https': 'https://pbs.twimg.com/media/D9tTwSJXUAEjiFU.png',
    'url': 'https://t.co/UJbFZk6FWm',
    'display_url': 'pic.twitter.com/UJbFZk6FWm',
    'expanded_url': 'https://twitter.com/cleantechnica/status/1142599021342707713/photo/1',
    't